## Install Required Libraries

In [1]:
!pip install nltk datasets transformers[torch] tokenizers evaluate rouge_score sentencepiece huggingface_hub -q

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 54.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 202

In [2]:
import transformers

print(transformers.__version__)

4.51.1


## Import Libraries

In [3]:
import nltk
import evaluate
import numpy as np
from datasets import load_dataset
from transformers import T5Tokenizer, DataCollatorForSeq2Seq
from transformers import T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer, EarlyStoppingCallback

2025-04-22 22:02:34.831047: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745359355.032092      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745359355.087552      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Load the `CNN/DailyMail` Dataset and `FLAN-T5` Model

- **Initialize Tokenizer and Model**

In [4]:
MODEL_NAME = "google/flan-t5-base"

tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

- **Load Data**

In [5]:
cnn_dailymail = load_dataset("cnn_dailymail", "3.0.0")

README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [6]:
cnn_dailymail

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

In [7]:
small_train = cnn_dailymail["train"].shuffle(seed=42).select(range(20000))
small_val = cnn_dailymail["validation"].shuffle(seed=42).select(range(2000))
small_test = cnn_dailymail["test"].shuffle(seed=42).select(range(2000))


dataset = {
    'train': small_train,
    'valid': small_val,
    'test': small_test
}

dataset

{'train': Dataset({
     features: ['article', 'highlights', 'id'],
     num_rows: 20000
 }),
 'valid': Dataset({
     features: ['article', 'highlights', 'id'],
     num_rows: 2000
 }),
 'test': Dataset({
     features: ['article', 'highlights', 'id'],
     num_rows: 2000
 })}

## Preprocess the Dataset

In [8]:
max_input_length = 512
max_target_length = 128

def preprocess_function(examples):
    inputs = ["summarize: " + doc for doc in examples["article"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, padding="max_length")

    # Tokenize the targets (highlights)
    labels = tokenizer(
        text_target=examples["highlights"],
        max_length=max_target_length,
        truncation=True,
        padding="max_length"
    )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply preprocessing to each split
tokenized_dataset = {
    split: ds.map(preprocess_function, batched=True)
    for split, ds in dataset.items()
}

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [9]:
# model.config.pad_token_id = tokenizer.pad_token_id

- **Define Data Collator**

## Load ROUGE Metric for Evaluation

In [10]:
from nltk.tokenize import sent_tokenize

nltk.download("punkt", quiet=True)
metric = evaluate.load("rouge")



def compute_metrics(eval_preds):
    preds, labels = eval_preds

    # Replace -100 with pad_token_id
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

    # Decode predictions and labels
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Format for ROUGE
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    # Compute ROUGE
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    result = {k: round(v * 100, 4) for k, v in result.items()}
    
    return result

## Define Training Arguments

In [11]:
# Global Parameters
L_RATE = 3e-4
BATCH_SIZE = 4
PER_DEVICE_EVAL_BATCH = 2
WEIGHT_DECAY = 0.01
SAVE_TOTAL_LIM = 2
NUM_EPOCHS = 5


training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    eval_strategy="steps",            
    eval_steps=500,                      
    logging_strategy="steps",               
    logging_steps=500,                     
    learning_rate=L_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=PER_DEVICE_EVAL_BATCH,
    weight_decay=WEIGHT_DECAY,
    save_total_limit=SAVE_TOTAL_LIM,
    num_train_epochs=NUM_EPOCHS,
    predict_with_generate=True,            
    push_to_hub=False,
    report_to="none"
)

In [12]:
# data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

- **Initialize Trainer**

In [13]:
trainer = Seq2SeqTrainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_dataset["train"],
   eval_dataset=tokenized_dataset["valid"],
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics
)

/tmp/ipykernel_19/1636498931.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


- **Train the Model**

In [14]:
trainer.train()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
500,1.448900,1.098987,24.429700,11.267600,19.968000,22.965000
1000,1.168900,1.103441,24.435100,11.227700,19.853500,23.012200
1500,1.194700,1.089318,25.123900,11.835800,20.586300,23.640000
2000,1.192800,1.084595,24.552800,11.506800,20.100100,23.012100
2500,1.178300,1.086169,24.105500,10.948300,19.640900,22.637900
3000,1.157000,1.081396,24.823600,11.809400,20.436900,23.380900
3500,1.166900,1.077635,24.173300,11.229200,19.849400,22.753500
4000,1.158400,1.087443,24.617300,11.572700,20.179000,23.152400
4500,1.159900,1.076188,24.701700,11.659200,20.305000,23.267100
5000,1.145200,1.072860,24.843400,11.613400,20.232700,23.415900


TrainOutput(global_step=25000, training_loss=0.9212205627441407, metrics={'train_runtime': 34962.8776, 'train_samples_per_second': 2.86, 'train_steps_per_second': 0.715, 'total_flos': 6.84757352448e+16, 'train_loss': 0.9212205627441407, 'epoch': 5.0})

In [15]:
trainer.save_model("./flan-t5-summarizer")
tokenizer.save_pretrained("./flan-t5-summarizer")

('./flan-t5-summarizer/tokenizer_config.json',
 './flan-t5-summarizer/special_tokens_map.json',
 './flan-t5-summarizer/spiece.model',
 './flan-t5-summarizer/added_tokens.json')

## Evaluation

In [16]:
trainer.evaluate()

{'eval_loss': 1.110196828842163,
 'eval_rouge1': 24.8565,
 'eval_rouge2': 11.4836,
 'eval_rougeL': 20.1841,
 'eval_rougeLsum': 23.2824,
 'eval_runtime': 501.4426,
 'eval_samples_per_second': 3.988,
 'eval_steps_per_second': 1.994,
 'epoch': 5.0}

In [17]:
trainer.evaluate(tokenized_dataset["test"])

{'eval_loss': 1.1009453535079956,
 'eval_rouge1': 25.3314,
 'eval_rouge2': 11.957,
 'eval_rougeL': 20.6773,
 'eval_rougeLsum': 23.8067,
 'eval_runtime': 500.1261,
 'eval_samples_per_second': 3.999,
 'eval_steps_per_second': 1.999,
 'epoch': 5.0}

- **Test the Model**

In [18]:
test_article = dataset["test"][0]["article"]
inputs = tokenizer("summarize: " + test_article, return_tensors="pt", truncation=True, padding="max_length", max_length=512).to(model.device)
summary_ids = model.generate(inputs.input_ids, max_length=200, num_beams=4, length_penalty=2.0, early_stopping=True)
print("Original Text:\n", test_article)
print("\n")
print("=="*30)
print("\n")
print("Generated Summary:\n", tokenizer.decode(summary_ids[0], skip_special_tokens=True))

Original Text:
 (CNN) I see signs of a revolution everywhere. I see it in the op-ed pages of the newspapers, and on the state ballots in nearly half the country. I see it in politicians who once preferred to play it safe with this explosive issue but are now willing to stake their political futures on it. I see the revolution in the eyes of sterling scientists, previously reluctant to dip a toe into this heavily stigmatized world, who are diving in head first. I see it in the new surgeon general who cites data showing just how helpful it can be. I see a revolution in the attitudes of everyday Americans. For the first time a majority, 53%, favor its legalization, with 77% supporting it for medical purposes. Support for legalization has risen 11 points in the past few years alone. In 1969, the first time Pew asked the question about legalization, only 12% of the nation was in favor. I see a revolution that is burning white hot among young people, but also shows up among the parents and g

## Inference

In [19]:
print(os.listdir('./results'))

NameError: name 'os' is not defined

In [ ]:
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
# import torch

# # Load the fine-tuned model and tokenizer from local directory
# model_path = "./results/checkpoint-1500"  # ✅ relative, not /kaggle/...

# tokenizer = AutoTokenizer.from_pretrained(model_path)
# model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

# def summarize(text, max_len=128):
#     # Prefix with "summarize:" for T5
#     input_text = "summarize: " + text.strip()

#     # Tokenize input
#     inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True)

#     # Generate summary
#     summary_ids = model.generate(
#         input_ids=inputs["input_ids"],
#         attention_mask=inputs["attention_mask"],
#         max_length=max_len,
#         num_beams=4,
#         early_stopping=True
#     )

#     # Decode and return summary
#     summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
#     return summary

# # Test it
# if __name__ == "__main__":
#     sample_text = """
#     Egypt is a country located in the northeastern corner of Africa, with a small part of its territory in Asia. It is known for its ancient civilization, with famous monuments such as the Giza pyramids, the Great Sphinx, and the temples of Luxor.
#     """
#     print("Summary:\n", summarize(sample_text))

In [ ]:
import os
import time


results_dir = './results' 

# List all subdirectories (assumes folders are named like 'checkpoint-1000', 'checkpoint-1500', etc.)
subdirs = [os.path.join(results_dir, subdir) for subdir in os.listdir(results_dir) if os.path.isdir(os.path.join(results_dir, subdir))]

# Sort subdirectories by the last modification time
subdirs.sort(key=lambda x: os.path.getmtime(x), reverse=True)

# Get the most recent checkpoint directory (first item in sorted list)
latest_checkpoint = subdirs[0] if subdirs else None

if latest_checkpoint:
    print(f"Loading model from the latest checkpoint: {latest_checkpoint}")
    
    # Load the tokenizer and model from the most recent checkpoint
    from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
    
    tokenizer = AutoTokenizer.from_pretrained(latest_checkpoint)
    model = AutoModelForSeq2SeqLM.from_pretrained(latest_checkpoint)
else:
    print("No checkpoints found.")

In [ ]:
from transformers import pipeline

# Create a summarization pipeline using the loaded model and tokenizer
summarizer = pipeline("summarization", model=model, tokenizer=tokenizer)

# Example input text to summarize
text = """
Kaggle is a platform for data science competitions and collaboration, 
where users can explore datasets, build models, and work with the latest AI tools. 
It provides a great opportunity for learning and sharing knowledge.
"""

# Generate summary
summary = summarizer(text, max_length=100, min_length=30, do_sample=False)

# Print result
print("Summary:", summary[0]['summary_text'])


In [ ]:
len(summary[0]['summary_text'])

In [ ]:
len(text)